In [ ]:
import requests
import json
import time
import uuid
import math
import itertools
from datetime import timedelta
from pathlib import Path
from typing import Dict, Generator, List, Tuple, Union

In [ ]:
class ModelClient:
    def __init__(
        self,
        api_key: str = "tpsg-MNvTQUAqUL84o4THLV1395IqTBIZHJJ",
        model: str = "gpt-4o-mini-2024-07-18",
        base_url: str = "https://api.tapsage.com",
        provider: str = "openai_chat_completion",
        max_tokens: int = 1024,
        temperature: float = 0.7,
    ):
        self.endpoint = f"{base_url}/api/v1/wrapper/{provider}/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        }
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = (
            "شما یک متخصص مجرب در حوزه‌های گردشگری، اقلیم‌شناسی و جغرافیای ایران هستید. "
            "تمام پاسخ‌های شما باید فقط به زبان فارسی معیار، دقیق، روان و مطابق با استانداردهای نگارش علمی و اطلاع‌رسانی ارائه شوند. "
            "به هیچ وجه در متن تولیدی خود از عبارات و کلمات زبان دیگری به غیر از فارسی استفاده نکنید."
        )

    def start(self, prompt: str) -> str:
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": prompt},
        ]
        payload = {
            "model": self.model,
            "messages": messages,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
        }
        resp = requests.post(self.endpoint, json=payload, headers=self.headers, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        return data["choices"][0]["message"]["content"].strip()

In [ ]:
assistant = ModelClient()

In [ ]:
class DataGenerator:
    PROMPT_TPL = (
        "\nشما نقش یک کارشناس خبره در حوزهٔ گردشگری، اقلیم‌شناسی و جغرافیای ایران را دارید. اطلاعات زیر مربوط به یک «{d_fa}» واقع در استان {prov} است. "
        "بر پایهٔ همهٔ جزئیات، حداکثر در ۱۰۰ واژه، یک پاراگراف دقیق، روان و کاملاً به زبان فارسی معیار بنویس که تمام داده‌های موجود (از جمله نام، موقعیت جغرافیایی یا طبقه‌بندی، "
        "ویژگی‌های شاخص، اعداد، سال‌ها و هر نکتهٔ قابل توجه) را به صورت یکپارچه و منسجم در بر گیرد. "
        "**اگر اطلاعات معتبر و مرتبطی دربارهٔ این مکان می‌دانی، می‌توانی آن را هم اضافه کنی، به شرطی که متن از ۱۰۰ واژه بیشتر نشود.** "
        "خروجی فقط باید همان یک پاراگراف باشد و هیچ متن اضافی، عنوان یا توضیحی خارج از آن تولید نشود.\n\n"
        "دادهٔ ساخت‌یافته:\n{struct}\n"
    )

    DOM_FA = {
        "geographical_feature": "ویژگی جغرافیایی",
        "natural_resources": "منبع طبیعی",
        "topography": "ویژگی توپوگرافی",
        "tourist_attraction": "جاذبهٔ گردشگری",
    }

    def __init__(
            self,
            assistant,
            data_dir: Union[str, Path] = "data",
            output_dir: Union[str, Path] = "province_texts",
            num_output: int = 5,
            progress_step: int = 1,
    ):
        self.assistant = assistant
        self.data_dir   = Path(data_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.num_output    = num_output
        self.progress_step = progress_step
        self._spinner = itertools.cycle("⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")
        self._start_time = None

    def _load(self, p: Path) -> Dict:
        with p.open("r", encoding="utf-8") as f:
            return json.load(f)

    def _iter_records_old(self, d: Dict) -> Generator[Dict, None, None]:
        for b in d.get("geographical_features", []):
            for it in b.get("description", []):
                yield {"domain": "geographical_feature", "subcategory": b.get("category"), **it}
        for k in ("natural_resources", "topography"):
            for it in d.get(k, []):
                yield {"domain": k, **it}
        for it in d.get("tourist_attractions", []):
            yield {"domain": "tourist_attraction", **it}

    def _flatten(self, node, domain, parent_sub=None):
        if isinstance(node, dict):
            desc = node.get("description")
            if isinstance(desc, list):
                sub = node.get("category") or node.get("name") or parent_sub
                for item in desc:
                    yield from self._flatten(item, domain, sub)
            else:
                rec = dict(node)
                rec["domain"] = domain
                if parent_sub and "subcategory" not in rec:
                    rec["subcategory"] = parent_sub
                yield rec
        elif isinstance(node, list):
            for item in node:
                yield from self._flatten(item, domain, parent_sub)

    def _iter_records_new(self, d: Dict) -> Generator[Dict, None, None]:
        for blk in d.get("geographical_features", []):
            yield from self._flatten(blk, "geographical_feature")
        for blk in d.get("natural_resources", []):
            yield from self._flatten(blk, "natural_resources")
        for blk in d.get("topography", []):
            yield from self._flatten(blk, "topography")
        yield from self._flatten(d.get("tourist_attractions", []), "tourist_attraction")

    def _iter_records(self, d: Dict) -> Generator[Dict, None, None]:
        try:
            for rec in list(self._iter_records_old(d)):
                yield rec
        except Exception:
            for rec in self._iter_records_new(d):
                yield rec

    def _prompt(self, rec: Dict, prov: str) -> str:
        clean = {k: v for k, v in rec.items() if k not in ("images", "vote")}
        struct = json.dumps(clean, ensure_ascii=False, indent=2)
        return self.PROMPT_TPL.format(
            d_fa=self.DOM_FA.get(rec["domain"], rec["domain"]), prov=prov, struct=struct
        )

    def _format_time(self, seconds: float) -> str:
        return str(timedelta(seconds=int(seconds)))

    def _progress(self, idx: int, total: int):
        if idx % self.progress_step != 0 and idx != total:
            return
        elapsed = time.time() - self._start_time
        speed = idx / elapsed if elapsed else 0
        eta = (total - idx) / speed if speed else math.inf
        bar_len = 30
        filled = int(bar_len * idx / total)
        bar = "█" * filled + "░" * (bar_len - filled)
        pct = idx * 100 / total
        msg = (
            f"\r{next(self._spinner)} |{bar}| "
            f"{idx}/{total} {pct:5.1f}% "
            f"| elapsed {self._format_time(elapsed)} "
            f"| speed {speed:5.2f} rec/s "
            f"| eta {self._format_time(eta)} "
        )
        print(msg, end="", flush=True)

    def run(self):
        for fp in sorted(self.data_dir.glob("*.json")):
            prov = fp.stem
            data = self._load(fp)
            records = list(self._iter_records(data))
            total = len(records)
            results = []
            self._start_time = time.time()
            self._spinner = itertools.cycle("⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")

            for idx, rec in enumerate(records, 1):
                self._progress(idx, total)
                prompt = self._prompt(rec, prov)
                try:
                    txt = self.assistant.start(prompt)
                except Exception as e:
                    print(f"\n✘ {prov} record {idx} failed: {e}")
                    continue
                results.append(
                    {
                        "id": str(uuid.uuid4()),
                        "province": prov,
                        "record_meta": {
                            "domain": rec["domain"],
                            "name": rec.get("name") or rec.get("subcategory"),
                        },
                        "prompt": prompt,
                        "text": txt,
                    }
                )
                if idx % self.num_output == 0:
                    tmp_path = self.output_dir / f"{prov}__checkpoint.json"
                    with tmp_path.open("w", encoding="utf-8") as f:
                        json.dump(results, f, ensure_ascii=False, indent=2)
                    print(f"\n💾 checkpoint {idx}/{total} → {tmp_path}")

            out_path = self.output_dir / f"{prov}.json"
            with out_path.open("w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 {prov}: saved {len(results)} records → {out_path}")

        print("\n✅ all provinces processed")

In [ ]:
DataGenerator(assistant).run()

⠼ |█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 5/114   4.4% | elapsed 0:00:18 | speed  0.27 rec/s | eta 0:06:39 
💾 checkpoint 5/114 → province_texts/اصفهان__checkpoint.json
⠏ |██░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 10/114   8.8% | elapsed 0:00:37 | speed  0.26 rec/s | eta 0:06:34 
💾 checkpoint 10/114 → province_texts/اصفهان__checkpoint.json
⠼ |███░░░░░░░░░░░░░░░░░░░░░░░░░░░| 15/114  13.2% | elapsed 0:00:55 | speed  0.27 rec/s | eta 0:06:05 
💾 checkpoint 15/114 → province_texts/اصفهان__checkpoint.json
⠏ |█████░░░░░░░░░░░░░░░░░░░░░░░░░| 20/114  17.5% | elapsed 0:01:12 | speed  0.28 rec/s | eta 0:05:40 
💾 checkpoint 20/114 → province_texts/اصفهان__checkpoint.json
⠼ |██████░░░░░░░░░░░░░░░░░░░░░░░░| 25/114  21.9% | elapsed 0:01:29 | speed  0.28 rec/s | eta 0:05:19 
💾 checkpoint 25/114 → province_texts/اصفهان__checkpoint.json
⠏ |███████░░░░░░░░░░░░░░░░░░░░░░░| 30/114  26.3% | elapsed 0:01:46 | speed  0.28 rec/s | eta 0:04:59 
💾 checkpoint 30/114 → province_texts/اصفهان__checkpoint.json
⠼ |█████████░░░░░░░░░░░░

In [ ]:
class QuestionGenerator:
    PROMPT_TPL = (
        "\nشما یک طراح حرفه‌ای پرسش‌های مطالعات اجتماعی هستید. با توجه به متن زیر دربارهٔ استان {prov}، "
        "دقیقاً پنج «پرسش و پاسخ» بنویس. مهم است که **تعداد آن‌ها دقیقاً پنج باشد (نه کمتر و نه بیشتر)**. "
        "هر پرسش باید به‌طور مستقیم به بخشی از متن مربوط باشد و پاسخ آن نیز **عین عبارت یا جملهٔ موجود در متن** باشد. "
        "**در متن پرسش باید به‌روشنی مشخص باشد که پرسش دربارهٔ چه موضوع یا بخش مشخصی از استان است.** "
        "تحت هیچ شرایطی کمتر از پنج «پرسش و پاسخ» تولید نکن.\n"
        "خروجی را دقیقاً در قالب زیر تولید کن (بدون هیچ متن یا توضیح اضافی):\n"
        "- پرسش: ...\n  پاسخ: ...\n"
        "- پرسش: ...\n  پاسخ: ...\n"
        "- پرسش: ...\n  پاسخ: ...\n"
        "- پرسش: ...\n  پاسخ: ...\n"
        "- پرسش: ...\n  پاسخ: ...\n\n"
        "متن:\n{text}\n"
    )

    def __init__(
        self,
        assistant: ModelClient,
        input_dir: Union[str, Path] = "province_texts",
        output_dir: Union[str, Path] = "province_questions",
        num_output: int = 5,
        progress_step: int = 1,
    ):
        self.assistant = assistant
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.num_output = num_output
        self.progress_step = progress_step
        self._spinner = itertools.cycle("⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")
        self._start_time = None

    def _format_time(self, sec: float) -> str:
        return str(timedelta(seconds=int(sec)))

    def _progress(self, idx: int, total: int):
        if idx % self.progress_step != 0 and idx != total:
            return
        elapsed = time.time() - self._start_time
        speed = idx / elapsed if elapsed else 0
        eta = (total - idx) / speed if speed else math.inf
        bar_len = 30
        filled = int(bar_len * idx / total)
        bar = "█" * filled + "░" * (bar_len - filled)
        pct = idx * 100 / total
        print(
            f"\r{next(self._spinner)} |{bar}| {idx}/{total} {pct:5.1f}% "
            f"| elapsed {self._format_time(elapsed)} "
            f"| speed {speed:5.2f} rec/s "
            f"| eta {self._format_time(eta)} ",
            end="",
            flush=True,
        )

    def _prompt(self, text: str, prov: str) -> str:
        return self.PROMPT_TPL.format(text=text, prov=prov)

    def run(self):
        for fp in sorted(self.input_dir.glob("*.json")):
            prov = fp.stem
            records = json.loads(fp.read_text(encoding="utf-8"))
            total = len(records)
            results: List[Dict] = []
            self._start_time = time.time()
            self._spinner = itertools.cycle("⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")

            for idx, rec in enumerate(records, 1):
                self._progress(idx, total)
                prompt = self._prompt(rec["text"], prov)
                try:
                    raw = self.assistant.start(prompt)
                except Exception as e:
                    print(f"\n✘ {prov} record {idx} failed: {e}")
                    continue

                qa_pairs = []
                lines = [l.strip() for l in raw.splitlines() if l.strip()]
                i = 0
                while i < len(lines) and len(qa_pairs) < 5:
                    if lines[i].startswith("-"):
                        q = lines[i].lstrip("-").strip()
                        a = ""
                        if i + 1 < len(lines):
                            nxt = lines[i + 1]
                            if nxt.lower().startswith(("پاسخ", "answer", "a:", "پاسخ:")):
                                a = nxt.split(":", 1)[-1].strip()
                                i += 1
                            else:
                                a = nxt
                                i += 1
                        qa_pairs.append({"question": q.replace("پرسش:", "").strip(), "answer": a})
                    i += 1
                while len(qa_pairs) < 5:
                    qa_pairs.append({"question": "", "answer": ""})

                results.append({**rec, "qa_pairs": qa_pairs})

                if idx % self.num_output == 0:
                    tmp = self.output_dir / f"{prov}__checkpoint.json"
                    tmp.write_text(json.dumps(results, ensure_ascii=False, indent=2), encoding="utf-8")
                    print(f"\n💾 checkpoint {idx}/{total} → {tmp}")

            out_path = self.output_dir / f"{prov}.json"
            out_path.write_text(json.dumps(results, ensure_ascii=False, indent=2), encoding="utf-8")
            print(f"\n💾 {prov}: saved {len(results)} records → {out_path}")

        print("\n✅ question generation complete")

In [ ]:
QuestionGenerator(assistant).run()

⠼ |█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 5/114   4.4% | elapsed 0:00:19 | speed  0.26 rec/s | eta 0:06:57 
💾 checkpoint 5/114 → province_questions/اصفهان__checkpoint.json
⠏ |██░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 10/114   8.8% | elapsed 0:00:44 | speed  0.23 rec/s | eta 0:07:38 
💾 checkpoint 10/114 → province_questions/اصفهان__checkpoint.json
⠼ |███░░░░░░░░░░░░░░░░░░░░░░░░░░░| 15/114  13.2% | elapsed 0:01:14 | speed  0.20 rec/s | eta 0:08:14 
💾 checkpoint 15/114 → province_questions/اصفهان__checkpoint.json
⠏ |█████░░░░░░░░░░░░░░░░░░░░░░░░░| 20/114  17.5% | elapsed 0:01:41 | speed  0.20 rec/s | eta 0:07:57 
💾 checkpoint 20/114 → province_questions/اصفهان__checkpoint.json
⠼ |██████░░░░░░░░░░░░░░░░░░░░░░░░| 25/114  21.9% | elapsed 0:02:11 | speed  0.19 rec/s | eta 0:07:49 
💾 checkpoint 25/114 → province_questions/اصفهان__checkpoint.json
⠏ |███████░░░░░░░░░░░░░░░░░░░░░░░| 30/114  26.3% | elapsed 0:02:42 | speed  0.18 rec/s | eta 0:07:34 
💾 checkpoint 30/114 → province_questions/اصفهان__checkpoint.json
